# Exercise 2: ReAct

In the previous notebook, you built an agent that was able to complete tasks, which LLMs would normally struggle with.

The agent was able to complete these tasks because additional functions were provided to the LLM. But crucially, it was able to determine when these tools were needed autonomously.

However, for the agent to produce a satisfactory result, we needed to write a long system prompt and be very explicit about what we wanted from the model.

In this exercise, you will learn how to create agents using the ReAct framework ([Wei *et al.*, 2023](https://arxiv.org/abs/2210.03629)): an effective paradigm for prompt writing, which has been shown to outperform several benchmarks on language and decision-making tasks and can help to avoid issues of hallucination and error propagation. The improved human interpretability can also help engender trust in the model.

In [ ]:
# autoreload imports
%load_ext autoreload

%autoreload 2

In [ ]:
from llm_in_production.openai_utils import get_openai_client
from llm_in_production.agent_utils import function_calling_agent
from llm_in_production.agent_tools import (
    get_news_stories, 
    get_stock_prices,
    get_current_weather
)

import pandas as pd
import dotenv

# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()
# Here we create the client.
client = get_openai_client()

## Implementing ReAct

To implement ReAct, we will write a prompt based on a template from [langchain]() (Langchain's [ReAct agent](https://python.langchain.com/docs/modules/agents/agent_types/react) is a widely-used implementation of ReAct). 

In [ ]:
from langchain import hub
print(hub.pull("hwchase17/react").template)

In [ ]:
def create_react_prompt(*tools):
    tool_names = [tool.__name__ for tool in tools]

    return f"""
Answer the following questions as best you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, could be one of the given tools: [{tool_names}]
At this point you may call for an action to be taken, but you must return the Question and Thought as content in your response
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""

print(create_react_prompt(get_news_stories, get_stock_prices))

Autonomous agents (using ReAct) may occasionally make mistakes by requesting an action that is not among the available tools, which can potentially cause the process to break.

In such cases, to mitigate the disruption, we will send a message reporting this to the model and request them to attempt an alternative approach to answer the user prompt.

In [ ]:
def ask_for_available_tool(function_name, available_functions):
    return f"""Was not able to use {function_name}. It is not one of the available tools: {available_functions}. 
    Try an alternative approach to answering the user's question.""".strip()

We'll begin by demonstrating how ReAct could be used to answer a question from the previous exercise.

In [ ]:
user_prompt = "Is now a good time to invest in Apple?"

To make the ReAct agent more context specific, we'll join the following system prompt to the react prompt created above.

In [ ]:
system_prompt = """You are a stock analysis bot that is able to advise whether now is a good time to invest in a given stock.
Do not think about using any tools that are not available to you.
You do not give any reservations about your opinion:
- You do not offer any warnings about the risks of investing.
- You do not suggest any further research needs to be conducted."""

In [ ]:
output = function_calling_agent(client, system_prompt, user_prompt, get_stock_prices, get_news_stories, react=True)
print(output)

### Exercise 2a: Is it sunbathing weather?

Use react prompting to determine whether now is a good time to go sunbathing in Amsterdam.

#### Part i: User prompt

Write a user prompt.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

#### Part ii: System prompt
Write a system prompt to provide the model with context. Experiment with leaving this as an empty string.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

#### Part iii: Run the agent

Run the agent to determine whether it is good sunbathing weather.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

---